# 

# Imports

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os, json


# dask bag
from dask import delayed
import dask.bag as db
import time
from dask.distributed import Client


# pre processing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, WordPunctTokenizer

# Maneggiare json

In [2]:
#path_to_json = 'smb://192.168.1.130/pdf_json/'
path_to_json = 'covid_distributed/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print(json_files[3])  

0098190ff26a17c7046fcccba55196195fac5281g.json


In [3]:
# load file 200
path = path_to_json + json_files[1] 
f = open(path)
data = json.load(f)

data['metadata']['authors'][0]#['affiliation']#['location']['country']

{'first': 'J',
 'middle': [],
 'last': 'Cohen',
 'suffix': '',
 'affiliation': {'laboratory': '',
  'institution': 'INRA, C.R.J. Domaine de Vilvert',
  'location': {'postCode': '78350',
   'settlement': 'Jouy-en-Josas',
   'country': 'France'}},
 'email': ''}

In [4]:
# Ottenere il testo del body text

N_section = len(data['body_text'])
text = str('')

for i in range(N_section):
    text = text + data['body_text'][i]['text']
    
text


'Rotaviruses, members of a genus of the Reoviridae family, possess a genome containing 11 segments of double-stranded RNA. The 11 segments of genomic RNA code for structural proteins found in viral particles and for nonstructural (NS53, NS35, NS34, and NS28) proteins found only in infected cells. Nucleotide sequences have been reported (for one or another rotavirus strain) for the RNA segments 4 to 11 (Both et al., 1982 (Both et al., , 1983a (Both et al., ,b, 1984 lmai eta/., 1983; Estes et a/., 1984; Kantharidis et al., 1987; Bremont et a/., 1987) . In this work molecular characterization of the bovine rotavirus RF strain has been continued and we report the complete nucleotide sequence of gene 1 of bovine rotavirus (RF strain) that codes for the internal structural protein VP1 Three types of rotaviral particles have been described. Complete infectious particles possess a double capsid. Removal of the outer proteins produces a single-shelled capsid that contains four,proteins: VP1 (12

# Dask bag

In [5]:
client = Client("tcp://192.168.1.125:8786")
client

<Client: 'tcp://192.168.1.125:8786' processes=3 threads=12, memory=18.18 GiB>

In [6]:
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]


In [7]:
len(json_files)

1000

In [8]:
path_to_json = 'file:///../../mnt/nfs/covid_distributed/'

In [9]:
# bag of single file

# Qui ci sono stati due grandi problemi da risolvere:
# quello che si fa in genere quando si lavora con i json e' usare un file txt nel quale ogni riga
# e' la trascrizione di un file json.
# Poi quello che si fa e' dare alla db.read_text() quel file.

# Quello che fa questa funzione e' dividere quindi il file ricevuto in righe utilizzando di default 
# il parametro linedelimiter = None.

# Questo e' un problema, perche' noi non abbiamo un singolo file! 
# Se noi provassimo a caricare un json la bag lo suddividerebbe nelle parole di cui e' composto
# perche' si aspetta un txt organizzato come detto in precedenza.

# Il modo di farle capire che vogliamo che legga il file come un'unica frase e' dirle che ogni
# frase di quel file e' divisa da una sequenza di lettere impossibile da trovare (linedelimiter='aabbccddff')
# in questo modo legge un file come un unico pezzo.

b = db.read_text(path_to_json + json_files[3]).map(json.loads)
b.take(1)[0]

{'paper_id': '0098190ff26a17c7046fcccba55196195fac5281',
 'metadata': {'title': 'Winter School on sEMG Signal Processing: An Initiative to Reduce Educational Gaps and to Promote the Engagement of Physiotherapists and Movement Scientists With Science',
  'authors': [{'first': 'Roberto',
    'middle': [],
    'last': 'Merletti',
    'suffix': '',
    'affiliation': {},
    'email': ''},
   {'first': 'Ales',
    'middle': [],
    'last': 'Holobar',
    'suffix': '',
    'affiliation': {},
    'email': ''},
   {'first': 'Francesco',
    'middle': [],
    'last': 'Negro',
    'suffix': '',
    'affiliation': {},
    'email': ''},
   {'first': 'Felipe',
    'middle': ['P'],
    'last': 'Carpes',
    'suffix': '',
    'affiliation': {'laboratory': 'Laboratory of Neuromechanics',
     'institution': 'Universidade Federal Do Pampa',
     'location': {'settlement': 'Uruguaiana', 'country': 'Brazil'}},
    'email': 'carpes@unipampa.edu.br'},
   {'first': 'Carlos',
    'middle': [],
    'last': 'D

# Caricare piu' json in una sola bag

In [10]:
start_morej = time.time()

# lista di indirizzi relativi ai file
l = []
for j in range(0,1000,1):
    l += [path_to_json + json_files[j]]

    
# Ora qui stiamo caricando 1000 file in un'unica bag
    
b = db.read_text(l).map(json.loads)

end_morej = time.time()

b
# Ora, come facciamo a considerare ad esempio i primi 3?

# Beh, si usa b.take(3), no? No.

# Per come funziona il take anche lui e' pensato per funzionare con il file txt, e di default
# non si aspetta che siano stati caricati piu file.
# Per questo motivo serve specificare quanti file abbiamo caricato e serve scrivere:

# b.take( m , npartitions = N)      m = numero json che vogliamo: i primi m files
#                                   N = numero file caricati

# Ci sono delle robe strane in come fa le partizioni, infatti ho scoperto che per funzionare non serve che
# N sia il numero di file caricati, ma basta che sia maggiore di m+1

# Quindi per prendere i primi 3 file di 1000 caricati basta fare:
# b.take( 3 , npartitions = 4)


# Se si mette un valore di m arbitrariamente grande il take prende tutti gli elementi ma non da errori

dask.bag<loads, npartitions=1000>

In [ ]:
data1 = b.take(1,npartitions=1)[0]
data1['body_text'][0]['text']

In [ ]:
futures = []

for j in range(0,1000,1):
    
    bb = db.read_text( path_to_json + json_files[j]).map(json.loads)
    
    futures.append(client.scatter(bb))

    
# Ora qui stiamo caricando 1000 file in un'unica bag

In [ ]:
bf = db.from_delayed(futures)

In [ ]:
#bf = bf.repartition(npartitions=22).persist()

In [ ]:
type(bf)

In [ ]:
data1 = bf.take(1)[0]
data1['body_text'][0]['text']

# Pre processing

In [ ]:
stop_words = stopwords.words('english')
more_stopwords = ['u', 'im', 'c', 'et', 'al']
stop_words = stop_words + more_stopwords

def isvalidword(x, stop_words = stop_words):
    return not x.lower() in stop_words

def isalpha(x):
    return x.isalpha()

def pre_process(words,bag=True):
    j = None 
    if(bag):
        j = words.pluck('body_text').flatten().pluck('text')
    else:
        j = db.from_sequence(words)
        
    j = j.map(WordPunctTokenizer().tokenize).flatten() 
    j = j.filter(isvalidword).filter(isalpha)
    
    return j

pre_process(['Talk aut aut joy, paper, scissor,water! trash.cozy jk 0 !;'],False).take(55)

jj=pre_process(b)

In [ ]:
def binop(t,x):
    return {"name":t["name"], "count":sum(da["count"] for da in (t,x))}


res=jj.map(lambda x: {"name":x,"count":1}).foldby("name", binop).map(lambda x:x[1])


res_top=res.topk(10,key=lambda x: x["count"]).compute() ##REPARTITION
res_top

In [ ]:
res_top_df = pd.DataFrame(res_top)
res_top_df

In [ ]:
x_values = [r["name"] for r in res_top]
y_values = [r["count"] for r in res_top]

fig,ax = plt.subplots(figsize=(22,9))
plt.bar( x_values , y_values , color ='maroon', width = 0.4)
plt.title('Most 10 frequent words in all papers', fontsize=30)
ax.tick_params(axis='both', which='major', labelsize=15)

plt.show()


# 1.3.2 - Country contribution

In [ ]:
# convert to a dataframe

# convert to a dataframe
def check(var, args):
    cc = var
    for i in range(len(args)):
        if args[i] in cc:
            cc = cc[args[i]]
            if cc=={}:
                return("")
        else:
            return("")
    return(cc.lower())

def denormalize(record):
    # returns a list for every nested item, every author of each paper
    res = []
    for author in record["metadata"]["authors"]:
        res.append({'title': record['metadata']['title'], 
                 'first': author['first'],
                 'middle': author['middle'],
                 'last': author['last'],
                 'suffix': author['suffix'],
                 'laboratory': check(author,['affiliation','laboratory']),
                 'institution': check(author,['affiliation','institution']),
                 'settlement': check(author,['affiliation','location','settlement']),
                 'country': check(author,['affiliation','location','country']),
                 'email': author['email']
               })
    return res
        


f = b.map(denormalize).flatten()
f.take(2, npartitions=5)[1]

In [ ]:
df = f.to_dataframe(meta={'title': str, 
                 'first': str,
                 'middle': str,
                 'last': str,
                 'suffix': str,
                 'laboratory': str,
                 'institution': str,
                 'settlement': str,
                 'country': str,
                 'email': str}).compute()
df.head()

In [ ]:
#df.groupby('country').count().sort_values('title').compute()
df[df.country != ''].country.value_counts().nlargest(20)

In [ ]:
#df.groupby('institution').count(split_out=8).compute() ##!!SPLIT OUT
df[df.institution != ''].institution.value_counts().nlargest(20)

# 1.3.3 - Title Embedding

In [ ]:
def nsplit(x):
    tit=x[0]
    res=[float(ri) for ri in x[1:]]
    return {"name":tit, "embed":res}

fname='file:///../../mnt/nfs/covid_distributed/wiki-news-300d-1M.vec'
datas= db.read_text(fname, blocksize='1MiB', encoding='utf-8', linedelimiter='\n')
model=datas.map(lambda x: x.split()).filter(lambda x: isvalidword(x[0])).map(nsplit)
#av_names=model.map(lambda x: x["name"].lower()).compute()
model=model.to_dataframe()
model=model.set_index("name", sorted=True)
model=client.persist(model)
#av_names=model.map(lambda x: x["name"].lower()).compute()
#av_names

In [ ]:
#model=model.to_dataframe()
#model = model.set_index('name')
#model = model.persist()

In [ ]:
%time model.loc['cat'].compute()

In [ ]:
# embdedding of a list of words and also this do a preprocess on the sentence

def embedding(sentence, model=model):
    emb = []
    for s in sentence:
        s = s.lower()
        if (s.isalpha()) and (not s in stop_words) and (s in model.index):
            res=model.loc[s].compute()
            #if len(res)!=0:
            emb.append(np.array(res[0]["embed"]))  # try to use compute
    if(len(emb)==0):                                                                        # insetad of take
        emb.append(np.ones(300))
    return emb

In [ ]:
type(b)

In [ ]:
# I create a df with paper_id and title

def denormalize2(x):
    return {
        'paper_id': x['paper_id'],
        'title': x['metadata']['title'],
        'title-embedding': embedding(x['metadata']['title'].split())
    }

f2 = b.map(denormalize2)
#print(f2.compute())
rre=f2.to_dataframe(meta={'paper_id' : str, 'title' : str, 'title-embedding':object})
rre
#embedding(rre[0])
#df2 = f2.to_dataframe(meta={'paper_id' : str, 'title' : str})
#df2.head()

In [ ]:
emb_df=rre.compute()

In [ ]:
emb_df

# Bonus point

In [ ]:
import sklearn
import joblib
joblib.parallel_backend('dask')

## or
#with joblib.parallel_backend('dask'):
def cosine_similarity(x,y):
    return np.dot(x,y)/( np.linalg.norm(x)*np.linalg.norm(y) )

In [ ]:
# Two problems here 

# 1) there are titles which are not in english => we must remove them
# 2) embedded titles are vectors of vectors with different lenght (different number of words), so we cannot use 
#    cosine similarity. For this reason for each title we sum the embedding of each word, obtaining one vector
#    of size 300 for each title.


clean_rre = rre.map(lambda x: {"title":x["title"], "vec":np.mean(x["title-embedding"], axis=0)}).compute()
clean_rre

In [ ]:
#clean_emb=np.array([clean_emb_del[i].compute() for i in range(len(clean_emb_del))])
M = sklearn.metrics.pairwise_distances([ce["vec"] for ce in clean_rre], metric=cosine_similarity)

In [ ]:
# rimuoviamo elementi dalla diagonale, sappiamo gia' che loro sono 1
np.fill_diagonal(M,0)

# cerchiamo indici del massimo della cosine similarity
ind = np.unravel_index(np.argsort(M, axis=None)[-15:-1][::-1], M.shape)
ind

In [ ]:
# Confusion matrix of cosine similarity

xtitles = list( [ce['title'] for ce in clean_rre] )[ind[0][0]-5:ind[0][0]+6]
ytitles = list( [ce['title'] for ce in clean_rre] )[ind[1][0]-5:ind[1][0]+6]

xticks = np.arange(0,11,1)
yticks = np.arange(11,22,1)

# mask to have only half confusion matrix
mask = np.triu(np.ones_like(M[ind[0][0]-5:ind[0][0]+6, ind[1][0]-5:ind[1][0]+6], dtype=bool), k=1)
# color map
cmap = sns.diverging_palette(220, 20, as_cmap=True)

f, ax = plt.subplots(figsize=(12, 10))
ax.set_title('Confusion Matrix of Cosine Similarity', fontsize=25)
sns.heatmap(M[ind[0][0]-5:ind[0][0]+6, ind[1][0]-5:ind[1][0]+6], square=True, linewidths=.5, 
cbar_kws={"shrink": .8}, cmap=cmap, xticklabels=xticks, yticklabels=yticks);

leg = pd.DataFrame(xtitles + ytitles, columns=['Paper title legend'])
leg

In [ ]:
print(start_morej-time.time())